In [1]:
import numpy as np
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtWidgets
from time import sleep
import pyvista as pv

pi = np.pi

pg.mkQApp()

## make a widget for displaying 3D objects
import pyqtgraph.opengl as gl
view = gl.GLViewWidget()

In [ ]:
view.clear()

path = "Bone.obj"
reader = pv.get_reader(path)
mesh = reader.read()
tri_mesh = mesh.triangulate()
faces = tri_mesh.faces.reshape(-1, 4)[:, 1:]


grid = gl.GLGridItem()
data = gl.GLMeshItem(meshdata=gl.MeshData(vertexes=mesh.points, faces=faces), drawEdges=True)

view.addItem(data)
view.addItem(grid)

view.show()

t = 0
def render():
    pass
    # global t
    # data.rotate(1, 1, 1, 0)
    
timer = QtCore.QTimer()
timer.timeout.connect(render)
timer.start(10)

if __name__ == '__main__':
    pg.exec()


In [ ]:
class Armature:
    def __init__(self, lengths: list, angles: list = None, dtype=np.float64):
        self.lengths = np.array(lengths, dtype=dtype)
        if angles is None:
            self.angles = np.zeros_like(self.lengths)
        else:
            self.angles = np.array(angles, dtype=dtype)

    def forward_solve(self, limit = None):
        global_angles = np.cumsum(self.angles)
        return np.sum([np.cos(global_angles[:limit]), np.sin(global_angles[:limit])]*self.lengths[:limit], axis=1)

    def set_angles(self, angles):
        self.angles = np.array(angles)

    def solve_2dof(self, goal, pole = [0, 0]):
        pole = np.array(pole)
        goal = np.array(goal)
        L1, L2 = self.lengths[:2]
        L3 = np.sqrt(np.sum(goal**2))

        if L3 >= L1 + L2:
            offset = pi if goal[0] < 0 else 0
            self.angles[0] = np.arctan(goal[1]/goal[0]) + offset
            self.angles[1] = 0
            return
        
        offset = np.arctan(goal[1]/goal[0])
        if goal[0] < 0:
            offset += pi

        self.angles[0] = np.arccos((L1**2+L3**2-L2**2)/(2*L1*L3)) + offset
        self.angles[1] = np.arccos((L2**2+L1**2-L3**2)/(2*L2*L1)) + pi

        if goal[0]*pole[1]-goal[1]*pole[0] < 0:
            self.angles[0] = self.angles[0] + 2 * (offset-self.angles[0])
            self.angles[1] *= -1

    def solve_3dof(self, goal, angle, pole = [0, 0]):
        goal = np.array(goal)
        joint2_pos = goal - self.lengths[2] * np.array([np.cos(angle), np.sin(angle)])
        self.solve_2dof(joint2_pos, pole=pole)
        self.angles[2] = angle - (self.angles[0] + self.angles[1])
    def __str__(self):
        return f"Armature: Lengths: {self.lengths}, Angles: {self.angles}"


class ArmaturePlotElement(Armature):
    def __init__(self, plot: pg.PlotWidget, lengths: list, angles: list = None, colors: list = ['r', 'g', 'b'], width=1):
        super().__init__(lengths, angles)

        self.colors = colors
        self.lines = []

        pos = np.array([0, 0])
        global_angle = 0
        for i, (length, local_angle, color) in enumerate(zip(self.lengths, self.angles, self.colors)):
            global_angle += local_angle
            end = pos + np.array((np.cos(global_angle), np.sin(global_angle)))* length
            color = self.colors[i % len(self.colors)]
            line = pg.PlotDataItem([pos[0], end[0]], [pos[1], end[1]], pen=pg.mkPen(color, width=width))
            self.lines.append(line)
            plot.addItem(line)
            
            pos = end

    def render(self):
        pos = np.array([0, 0])
        global_angle = 0
        for length, local_angle, line in zip(self.lengths, self.angles, self.lines):
            global_angle += local_angle
            end = pos + np.array((np.cos(global_angle), np.sin(global_angle)))* length
            # line = pg.PlotDataItem([pos[0], end[0]], [pos[1], end[1]], pen=pg.mkPen(color))
            line.setData([pos[0], end[0]], [pos[1], end[1]])
            pos = end




In [ ]:
target = [2, 0]

plot = pg.plot()
plot.setXRange(-5, 5)
plot.setYRange(-5, 5)
plot.setWindowTitle('Arm')

arm = ArmaturePlotElement(plot, [3, 1, .5], width=10)
arm.solve_2dof(target)
target_element = pg.CrosshairROI(target, 10)
target_element.scale(0.1)
pole_element = pg.TargetItem()


plot.addItem(target_element)
plot.addItem(pole_element)

index = 0
def render():
    arm.solve_3dof(target_element.pos(), target_element.angle()/(180/pi), pole=pole_element.pos())

    arm.render()

    
timer = QtCore.QTimer()
timer.timeout.connect(render)
timer.start(10)

if __name__ == '__main__':
    pg.exec()
